## Examples and applications

During this lesson, we'll explore some variational algorithm examples and how to apply them:

- How to write a custom variational algorithm
- How to apply a variational algorithm to find minimum eigenvalues
- How to utilise variational algorithms to solve application use cases

### Problem definitions

Imagine that we want to use a variational algorithm to find the eigenvalue of the following observable:

$$
\hat{O}_1 = 2 II - 2 XX + 3 YY - 3 ZZ,
$$

This observable has the following eigenvalues:

$$
\left\{
\begin{array}{c}
\lambda_0 = -6 \\
\lambda_1 = 4 \\
\lambda_2 = 4 \\
\lambda_3 = 6
\end{array}
\right\}
$$

And eigenstates:

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
\end{array}
\right\}
$$

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

## Custom VQE

We'll first explore how to construct a VQE instance manually to find the lowest eigenvalue for $\hat{O}_1$. This will incorporate a variety of techniques that we have covered throughout this course. 

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values

We can use this cost function to calculate optimal parameters

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(8)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

Finally, we can use our optimal parameters to calculate our minimum eigenvalues:

In [ ]:
observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

In [ ]:
from numpy.linalg import eigvalsh

solution_eigenvalue = min(eigvalsh(observable.to_matrix()))
print(
    f"Percent error: {abs((experimental_min_eigenvalue - solution_eigenvalue)/solution_eigenvalue):.2e}"
)

As you can see, the result is extremely close to the ideal. 

## Configuring Qiskit's VQE

For convenience, we can also use existing Qiskit [VQE](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.html) implementations to find the lowest eigenvalue of the first observable $\hat{O}_1$ and look at all the output results.

In this case we will use the following:

- We'll start exploring reference operator $\equiv I$ to demonstrate the speed increases this could provide
- Qiskit Terra's [`Estimator`](https://qiskit.org/documentation/stubs/qiskit.primitives.Estimator.html#qiskit.primitives.Estimator)
- [`SLSQP`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.SLSQP.html) (i.e. Sequential Least SQuares Programming) optimizer
- Additionally, we will set the initial point for the `SLSQP` optimizer to $\vec\theta_0 = (1, \cdots, 1)$.

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
import numpy as np

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)

vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))

Now that we have initialized the `VQE` instance, we can get the results with the [`VQE.compute_minimum_eigenvalue`](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue.html#qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue) method. Let us look at the results.

In [ ]:
result = vqe.compute_minimum_eigenvalue(observable)
print(result)

Let us look at the `optimizer_result`:

In [ ]:
print(result.optimizer_result)

Of all this information, however, the most important part is the eigenvalue. Let us compare it with the theoretical value:

In [ ]:
from numpy.linalg import eigvalsh

eigenvalues = eigvalsh(observable.to_matrix())
min_eigenvalue = eigenvalues[0]

print("EIGENVALUES:")
print(f"  - Theoretical: {min_eigenvalue}.")
print(f"  - VQE: {result.eigenvalue}")
print(
    f"Percent error >> {abs((result.eigenvalue - min_eigenvalue)/min_eigenvalue):.2e}"
)

As you can see, the result is extremely close to the ideal.

However, we still haven't looked at the eigenstates, since they were not part of `results`. For this purpose, let us bind the optimal parameter values `result.optimal_parameters` to `results.optimal_circuit` and define a [`Statevector`](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Statevector.html) from that bound (i.e. non-parametrized) circuit.

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)
optimal_vector = Statevector(optimal_circuit)

rounded_optimal_vector = np.round(optimal_vector.data, 3)
print(f"EIGENSTATE: {rounded_optimal_vector}")

This result does not seem too close to the theoretical one of $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) \equiv [\frac{1}{\sqrt{2}},0,0,\frac{1}{\sqrt{2}}]$. However, notice that eigenvectors are defined up to a constant factor. Furthermore, quantum states are always normalized and equivalent up to a global phase, so we can easily verify that these two state vectors are equivalent.

In [ ]:
from numpy.linalg import eigh

_, eigenvectors = eigh(observable.to_matrix())
min_eigenvector = eigenvectors.T[0]  # Note: transpose to extract by index

optimal_vector.equiv(min_eigenvector, atol=1e-4)

We can conclude that the state we obtained is equivalent to the ideal one up to $10^{-4}$.

### Add reference state

In the previous example we have not used any reference operator $U_R$. Now let us think about how the ideal eigenstate $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ can be obtained. Consider the following circuit.

In [ ]:
from qiskit import QuantumCircuit

ideal_qc = QuantumCircuit(2)
ideal_qc.h(0)
ideal_qc.cx(0, 1)

ideal_qc.draw("mpl")

We can quickly check that this circuit gives us the desired state.

In [ ]:
Statevector(ideal_qc)

Now that we have seen how a circuit preparing the solution state looks like, it seems reasonable to use a Hadamard gate as a reference circuit, so that the full ansatz becomes:

In [ ]:
reference = QuantumCircuit(2)
reference.h(0)
# Include barrier to separate reference from variational form
reference.barrier()

ref_ansatz = ansatz.decompose().compose(reference, front=True)

ref_ansatz.draw("mpl")

For this new circuit, the ideal solution could be reached with all the parameters set to $0$, so this confirms that the choice of reference circuit is reasonable.

Now let us compare the number of cost function evaluations, optimizer iterations and time taken with those of the previous attempt.

In [ ]:
num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print("NO REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
# You can change the ansatz of the already defined vqe object instead of creating a new one
vqe.ansatz = ref_ansatz

ref_result = vqe.compute_minimum_eigenvalue(observable)

In [ ]:
num_evaluations_ref = ref_result.cost_function_evals
num_iterations_ref = ref_result.optimizer_result.nit
time_ref = ref_result.optimizer_time

print("ADDED REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations_ref}")
print(f"  - Number of iterations: {num_iterations_ref}")
print(f"  - Time: {time_ref:.5f} seconds")
print()

if num_evaluations_ref < num_evaluations:
    print(">> Number of cost function evaluations improved")
elif num_evaluations_ref > num_evaluations:
    print(">> Number of cost function evaluations worsened")
if num_iterations_ref < num_iterations:
    print(">> Number of iterations improved")
elif num_iterations_ref > num_iterations:
    print(">> Number of iterations worsened")
if time_ref < time:
    print(">> Time improved")
elif time_ref > time:
    print(">> Time worsened")

### Change initial point

Now that we have seen the effect of adding the reference state, we will go into what happens when we choose different initial points $\vec{\theta_0}$. In particular we will use $\vec{\theta_0}=(0,0,0,0,6,0,0,0)$ and $\vec{\theta_0}=(6,6,6,6,6,6,6,6,6)$. Remember that, as discussed when the reference state was introduced, the ideal solution would be found when all the parameters are $0$, so the first initial point should give fewer evaluations, iterations and time.

In [ ]:
vqe.initial_point = [0, 0, 0, 0, 6, 0, 0, 0]

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
vqe.initial_point = 6 * np.ones(8)

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

## VQD example

Now instead of looking for only the lowest eigenvalue of our observables, we will look for all $4$. Following the notation from the previous chapter (as well as that from Qiskit's [VQD](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.html) class), that means $k=4$.

Remember that the cost functions of VQD are:

$$
C_{l}(\vec{\theta}) := 
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\sum_{j=0}^{l-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2 
\quad \forall l\in\{1,\cdots,k\}=\{1,\cdots,4\}
$$

This is particularly important because a vector $\vec{\beta}=(\beta_0,\cdots,\beta_{k-1})$ (in this case $(\beta_0, \beta_1, \beta_2, \beta_3)$) must be passed as an argument when we define the `VQD` object.

Also, in Qiskit's implementation of VQD, instead of considering the effective observables described in the previous notebook, the fidelities $|\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2$ are calculated directly via the [`ComputeUncompute`](https://qiskit.org/documentation/stubs/qiskit.algorithms.state_fidelities.ComputeUncompute.html) algorithm, that leverages a `Sampler` primitive to sample the probability of obtaining $|0\rangle$ for the circuit
$U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})$. This works precisely because this probability is

$$
\begin{aligned}

p_0

& = |\langle 0|U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})|0\rangle|^2 \\[1mm]

& = |\big(\langle 0|U_A^\dagger(\vec{\theta})\big)\big(U_A(\vec{\theta^j})|0\rangle\big)|^2 \\[1mm]

& = |\langle \psi(\vec{\theta}) |\psi(\vec{\theta^j}) \rangle|^2 \\[1mm]

\end{aligned}
$$


Finally, to try out a new optimizer, let us use [`COBYLA`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.COBYLA.html) instead of `SLSQP`.

In [ ]:
from qiskit.primitives import Sampler
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.algorithms.eigensolvers import VQD

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 4
betas = [40, 60, 30, 30]

vqd = VQD(estimator, fidelity, ansatz, optimizer, k=k, betas=betas)

As the only API difference from the previous examples, notice that instead of calling the `VQE.compute_minimum_eigenvalue` method, we will call [`VQD.compute_eigenvalues`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.compute_eigenvalues.html). Let's start by examining the following observable:

$$
\hat{O}_2 := 2 II - 3 XX + 2 YY - 4 ZZ
$$

This observable has the following eigenvalues:

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

And eigenstates:

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)
\end{array}
\right\}
$$

In [ ]:
observable_2 = SparsePauliOp.from_list([("II", 2), ("XX", -3), ("YY", 2), ("ZZ", -4)])

result = vqd.compute_eigenvalues(observable_2)
print(result)

The [`VQDResult`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQDResult.html) we obtain is completely analogous to the `VQEResult`. It only differs in that each attribute is a list whose $i$-th element corresponds to the $i$-th eigenvalue.

Now that we have seen the eigenvalues, let us compare the experimental eigenvectors with the theoretical ones:

In [ ]:
optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

These results are the same as the expected ones except from a small approximation error and global phase.

Now let us solve this problem for the first observable $\hat{O}_1 := 2 II - 2 XX + 3 YY - 3 ZZ$.

In [ ]:
result = vqd.compute_eigenvalues(observable)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Here the eigenstates corresponding to $\lambda_1 = \lambda_2 = 4$ are not the same as the expected ones: 

$$
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle) \\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
$$

This happens precisely because as $\lambda_1=\lambda_2$, any complex linear combination is also a eigenstate with the same eigenvalue:

$$
\begin{aligned}
\alpha_1 |\phi_1\rangle + \alpha_2 |\phi_2\rangle
& = \frac{1}{\sqrt{2}}(\alpha_1 |00\rangle + \alpha_2 |01\rangle - \alpha_2 |10\rangle - \alpha_1 |11\rangle) \\[1mm]
& \equiv \frac{1}{\sqrt{2}}[\alpha_1, \alpha_2, -\alpha_2, -\alpha_1]
\end{aligned}
$$

That is exactly what we are seeing with these results.

### Change betas

As mentioned in the instances lesson, the values of $\vec{\beta}$ should be bigger than the difference between eigenvalues. Let us see what happens when they do not satisfy that condition with $\hat{O}_2$

$$
\hat{O}_2 = 2 II - 3 XX + 2 YY - 4 ZZ
$$

with eigenvalues

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

In [ ]:
vqd.betas = np.ones(4)
result = vqd.compute_eigenvalues(observable_2)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

This time, the optimizer returns the same state $|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ as a proposed solution to all eigenstates: which is clearly wrong. This happens because the betas were too small to penalize the minimum eigenstate in the successive cost functions. Therefore, it was not excluded from the effective search space in later iterations of the algorithm, and always chosen as the best possible solution.

We recommend experimenting with the values of $\vec{\beta}$, and ensuring they are bigger than the difference between eigenvalues. 

## Quantum Chemistry: Ground State and Excited Energy Solver

Our objective is to minimize the expectation value of the observable representing energy (Hamiltonian $\hat{\mathcal{H}}$):

$$
\min_{\vec\theta} \langle\psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle
$$

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()

mapper = JordanWignerMapper()

h2_reference_state = HartreeFock(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
)

ansatz = UCC(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
    initial_state=h2_reference_state,
    excitations=2,
)

ansatz.decompose().decompose().draw("mpl")

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Sampler, Estimator
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

estimator = Estimator()
sampler = Sampler()
fidelity = ComputeUncompute(sampler)

vqe = VQE(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)
gse = GroundStateEigensolver(qubit_mapper=mapper, solver=vqe)
result = gse.solve(h2_problem)

print(result)

We can also leverage VQD to solve for the excited states

In [ ]:
from qiskit.algorithms.eigensolvers import VQD

h2_operators, _ = gse.get_qubit_operators(h2_problem)

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 3
betas = [33, 33, 33]

vqd = VQD(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    fidelity=fidelity,
    initial_point=np.zeros(ansatz.num_parameters),
    k=k,
    betas=betas,
)
result = vqd.compute_eigenvalues(operator=h2_operators)
vqd_values = result.optimal_values
print(vqd_values)

## Optimization: Max-Cut

The maximum cut (Max-Cut) problem is a combinatorial optimization problem that involves dividing the vertices of a graph into two disjoint sets such that the number of edges between the two sets is maximized. More formally, given an undirected graph $G=(V,E)$, where $V$ is the set of vertices and $E$ is the set of edges, the Max-Cut problem asks to partition the vertices into two disjoint subsets, $S$ and $T$, such that the number of edges with one endpoint in $S$ and the other in $T$ is maximized.

We can apply Max-Cut to solve a various problems including: clustering, network design, phase transitions, etc. We'll start by creating a problem graph:

In [ ]:
import networkx as nx

n = 4
G = nx.Graph()
G.add_nodes_from(range(n))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
G.add_weighted_edges_from(edge_list)

colors = ["red" for i in range(n)]


def draw_graph(G, colors):
    """Draws the graph with the chose colors"""
    layout = nx.shell_layout(G)
    nx.draw_networkx(G, node_color=colors, pos=layout)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)


draw_graph(G, colors)

In order to comfortably access the weights of all the edges, you can use the adjacency matrix. This matrix can be obtained with [networkx.to_numpy_array()](https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.to_numpy_array.html).

In [ ]:
w = nx.to_numpy_array(G)
print(w)

The [`qiskit_optimization`](https://qiskit.org/documentation/optimization/index.html) module includes a class called [`Maxcut`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.html) that allows you to define a weighted Max-Cut problem from the adjacency matrix $w$. From that problem we can obtain the equivalent binary optimization problem we deduced in the previous section with [`Maxcut.to_quadratic_program()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.to_quadratic_program.html)

In [ ]:
from qiskit_optimization.applications import Maxcut

max_cut = Maxcut(w)

quadratic_program = max_cut.to_quadratic_program()
print(quadratic_program.prettyprint())

This problem can be expressed as a binary optimization problem. For each node $0 \leq i < n$, where $n$ is the number of nodes of the graph (in this case $n=4$), we will consider the binary variable $x_i$. This variable will have the value $1$ if node $i$ is one of the groups that we'll label $1$ and $0$ if it's in the other group, that we'll label as $0$. We will also denote as $w_{ij}$ (element $(i,j)$ of the adjacency matrix $w$) the weight of the edge that goes from node $i$ to node $j$. Because the graph is undirected, $w_{ij}=w_{ji}$. Then we can formulate our problem as maximizing the following cost function:

$$
\begin{aligned}
C(\vec{x})
& =\sum_{i,j=0}^n w_{ij} x_i(1-x_j)\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i,j=0}^n w_{ij} x_ix_j\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i=0}^n \sum_{j=0}^i 2w_{ij} x_ix_j
\end{aligned}
$$

To solve this problem with a quantum computer, we are going to express the cost function as the expected value of an observable. However, the observables that Qiskit admits natively consist of Pauli operators, that have eigenvalues $1$ and $-1$ instead of $0$ and $1$. That's why we are going to make the following change of variable:

Where $\vec{x}=(x_0,x_1,\cdots ,x_{n-1})$. We can use the adjacency matrix $w$ to comfortably access the weights of all the edge. This will be used to obtain our cost function:

$$
z_i = 1-2x_i \rightarrow x_i = \frac{1-z_i}{2}
$$

This implies that:

$$
\begin{array}{lcl} x_i=0 & \rightarrow & z_i=1 \\ x_i=1 & \rightarrow & z_i=-1.\end{array}
$$

So the new cost function we want to maximize is:

$$
\begin{aligned}
C(\vec{z})
& = \sum_{i,j=0}^n w_{ij} \bigg(\frac{1-z_i}{2}\bigg)\bigg(1-\frac{1-z_j}{2}\bigg)\\[1mm]

& = \sum_{i,j=0}^n \frac{w_{ij}}{4} - \sum_{i,j=0}^n \frac{w_{ij}}{4} z_iz_j\\[1mm]

& = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j
\end{aligned}
$$

Moreover, the natural tendency of a quantum computer is to find minima (usually the lowest energy) instead of maxima so instead of maximizing $C(\vec{z})$ we are going to minimize:

$$
-C(\vec{z}) =  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Now that we have a cost function to minimize whose variables can have the values $-1$ and $1$, we can make the following analogy with the Pauli $Z$:

$$
z_i \equiv Z_i = \overbrace{I}^{n-1}\otimes ... \otimes \overbrace{Z}^{i} \otimes ... \otimes \overbrace{I}^{0}
$$

In other words, the variable $z_i$ will be equivalent to a $Z$ gate acting on qubit $i$. Moreover:

$$
Z_i|x_{n-1}\cdots x_0\rangle = z_i|x_{n-1}\cdots x_0\rangle \rightarrow \langle x_{n-1}\cdots x_0 |Z_i|x_{n-1}\cdots x_0\rangle = z_i
$$

Then the observable we are going to consider is:

$$
\hat{H} = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} Z_iZ_j
$$

to which we will have to add the independent term afterwards:

$$
\texttt{offset} = - \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

This transformation can be done with [`QuadraticProgram.to_ising()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.QuadraticProgram.to_ising.html).

In [ ]:
observable, offset = quadratic_program.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(observable))

We can use a VQE instance to find the optimal parameters as follows:

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
import numpy as np

ansatz = TwoLocal(observable.num_qubits, "rx", reps=1)
optimizer = COBYLA()

vqe = VQE(
    estimator=Estimator(),
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)

result = vqe.compute_minimum_eigenvalue(observable)

print(result)

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)

x = max_cut.sample_most_likely(Statevector(optimal_circuit))
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", quadratic_program.objective.evaluate(x))

# plot results
colors = ["red" if x[i] == 0 else "orange" for i in range(n)]
draw_graph(G, colors)

With this lesson, you have learned:

- How to write a custom variational algorithm
- How to apply a variational algorithm to find minimum eigenvalues
- How to utilise variational algorithms to solve application use cases
 
Proceed to the final lesson to take your assessment and earn your badge!